<h2 align="center"> Gaussian Mixtre from Scratch (Math) </h2> 
<h3 align="center"> Author: Ibrahim O Alabi, PhDc </h3>

This notebook is part of my series on Introduction to Python for Data Science. This is my way of contributing to open source knowledge. If you find this content useful, please consider leaving a **star** on this [repository](https://github.com/Ibrahim-Ola/homemade-ML.git).

---

The Gaussian Mixture Model (GMM) is a technique that derives clusters from the likelihood perspective. In supervised learning classification setting, our goal was to estimate/learn the probability of the labels given the features (i.e. $\mathbb{P}(y|\textbf{x})$). However, in unsupervised learning, since there are no lables to predict, out task reduces to density estimation (i.e. $\mathbb{P}(\textbf{x})$). For GMM, we consider our data as a mixture of Gaussians. That is, our data is modeled as follows:

$$\mathbb{P}(\textbf{x}) = \sum_{k = 1}^K \pi_k \mathcal{N}(\textbf{x}| \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$$

with

$$\sum_{k = 1}^K \pi_k = 1, \quad \quad \pi_k \in [0,1] $$

Where $\mathcal{N}(\textbf{x}| \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$ is the multivariate normal density function with parameters $\boldsymbol{\mu}_k$ (mean vector) and $\boldsymbol{\Sigma}_k$ (covariance matrix). $\pi_k$ is the mixture coefficient which estimates the amount of contibution of each Gaussian to the mixture. Hence, the parameter of of the GMM are; $\Theta = \{\pi_k, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k, k = 1,2, \cdots, K\}$.

The likelihood of GMM is maximized using the Expectation Maximization approach, with the assumption that latent cluster assignment for each observation is known.

### Expectation Maximization (EM)

We mentioned previously that the likelihood of GMM could be simply maximized if the latent cluster assignment for observation is known. In practice, these latent values are never known (as a matter of fact, that is what we want from the analysis). Thus, we iterate over the following two steps:

**Step 1 (Expectation Step):** here, a $\texttt{soft}$ assignment of each observation ($\textbf{x}_i \in \mathcal{D}$) to a cluster ($z_i = k$) is made based on our best guess. By $\texttt{soft}$ assignment, we mean that a datapoint $\textbf{x}_i \in \mathcal{D}$ can belong to more than one cluster with varying degrees. The $\texttt{soft}$ assignment is achieved by computing the responsibility score for each $\textbf{x}_i \in \mathcal{D}$. The responsibility score ($r(\textbf{x},k) = \mathbb{P}(z = k|\textbf{x})$). The responsibility score for an observation $r(\textbf{x}_i,k)$ tells us how likely it is that the observation belongs to cluster $k$. The formula is:

$$r(\textbf{x},k) = \frac{\pi_k \mathcal{N}(\textbf{x}| \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}{\sum_{c = 1}^K \pi_c \mathcal{N}(\textbf{x}| \boldsymbol{\mu}_c, \boldsymbol{\Sigma}_c)}$$

It is important to note here that $r(\textbf{x},k)$ is a matrix with dimesnsions $(n \times k)$, where $n$ is the sample size and $k$ is the cluster size.

**Step 2 (Maximization Step):** here, we generalize the likelihood using the responsibility scores $r(\textbf{x},k)$ obtained in Step 1. With these responsibilities, the parameters $\Theta = \{\pi_k, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k, k = 1,2, \cdots, K\}$ are re-estimated. This invovles differentiating the likelihood function (involving responsibilities) and equating to zero (not shown). The resulting estimates of the parameters are;

$$\boldsymbol{\mu}_k = \frac{1}{N_k} \sum_{\textbf{x}_i\in \mathcal{D}} r(\textbf{x}_i,k) \textbf{x}_i, \qquad N_k = \sum_{\textbf{x}_i\in \mathcal{D}} r(\textbf{x}_i,k)$$

$$\boldsymbol{\Sigma}_k = \frac{1}{N_k} \sum_{\textbf{x}_i\in \mathcal{D}} r(\textbf{x}_i,k) (\textbf{x}_i- \boldsymbol{\mu}_k)(\textbf{x}_i- \boldsymbol{\mu}_k)^\top = \frac{1}{N_k} \sum_{\textbf{x}_i\in \mathcal{D}} r(\textbf{x}_i,k) (\textbf{x}_i- \boldsymbol{\mu}_k)\otimes(\textbf{x}_i- \boldsymbol{\mu}_k)$$

$$\pi_k = \frac{1}{|\mathcal{D}|} N_k$$

$\texttt{Note}$: $\otimes$ is called the outer product. The outer product of two vectors **u** and **v** is given as $\textbf{u}\otimes\textbf{v} = \textbf{u}\textbf{v}^\top$. I introduced this symbol because I used it in my implementation ($\texttt{numpy.outer}(\cdot)$). Also note that $N_k \neq |\mathcal{D}|$, it is the continuous version of the number of elements in a cluster or number of observations of a particular cluster. It is continuous because of the soft assignment.


### Convergence

Our EM algorithm has to stop at some point. When the difference between two subsequent likelihoods (we typically use log-liklihood) is lesser than a set tolerance, the algorithm terminates. The likelihood function of GMM is;

$$L(\Theta| \mathcal{D}) = \prod_{\textbf{x}_i \in \mathcal{D}}\sum_{k = 1}^K \pi_k \mathcal{N}(\textbf{x}| \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$$

Its log-likelihood is given as:

$$\mathcal{L}(\Theta| \mathcal{D}) = \sum_{\textbf{x}_i \in \mathcal{D}} \log \left(\sum_{k = 1}^K \pi_k \mathcal{N}(\textbf{x}| \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)\right)$$

#### A simple reflection on the Log-likelihood

The log-likelihood as written above might be undefined for some observations. As an example, if it is highly unlikely that an observation comes from any of the Gaussians, then, $\mathbb{P}(\textbf{x})$ will be very small or even equal to zero. In such cases, the the log with be undefined. To combact this issue, I have appealed to the "log-sum-exponential" trick. I used it directly from $\texttt{Scipy}$. Reference: [Scipy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html).

### Implementation from Scratch

To implement, I have written a Class called `GaussianMixModel`. The Class allows users to input, intial parameters $\Theta = \{\pi_k, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k, k = 1,2, \cdots, K\}$ of their choice. If it is left empty, the algorithm splits the supplied data into $k$ equal sub-parts, where $k$ is the number number of clusters to be formed. The means and covariances of these $k$ dinstinct sub-parts are used for parameter initialization. The mixure weight is initialized as $\frac{1}{k}$. Finally, $\mathcal{N}(\textbf{x}| \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$ is compted using the $\texttt{multivariate_normal.pdf}$ function from $\texttt{Scipy}$. See [Scipy documentation](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html).

## References

1. [Fundamentals of Machine Learning](https://cs.mcgill.ca/~wlh/comp451/schedule.html)